In [3]:
import gymnasium as gym
import numpy as np

is_training = True
render = False

episodes = 5
learning_rate_a = 0.9
discount_factor_g = 0.9

epsilon = 1
epsilon_decay_rate = 2/episodes
rng = np.random.default_rng()

env = gym.make("MountainCar-v0", render_mode='human' if render else None)
# Break POS and Velocity into 20 segments
pos_space = np.linspace(env.observation_space.low[0], env.observation_space.high[0], 20) 
vel_space = np.linspace(env.observation_space.low[1], env.observation_space.high[1], 20)

q = np.zeros((len(pos_space), len(vel_space), env.action_space.n))

for i in range(episodes):
    batch_best = -1001

    state = env.reset()[0]
    # find segment of agents POS and Velocity
    state_p = np.digitize(state[0], pos_space)
    state_v = np.digitize(state[1], vel_space)

    terminated = False
    rewards = 0

    while(not terminated and rewards > -1000):
        if is_training and rng.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q[state_p, state_v, :])

        new_state,reward,terminated,_,_ = env.step(action)
        new_state_p = np.digitize(state[0], pos_space)
        new_state_v = np.digitize(state[1], vel_space)

        if is_training:
            q[state_p, state_v, action] = q[state_p, state_v, action]+learning_rate_a*(
                reward+discount_factor_g*np.max(q[new_state_p,new_state_v,:])-q[state_p, state_v, action]
            )

        state = new_state
        print(f'{i} - state: {state}')
        state_p = new_state_p
        state_v = new_state_v

        rewards += reward

    epsilon = max(epsilon - epsilon_decay_rate, 0)
    
    batch_best = max(batch_best, rewards)
    if i % 100 == 0:
        print(f'Episode: {i+1} - Best of the batch: {batch_best}')
        batch_best = -1001
env.close()



0 - state: [-0.44983503 -0.00156013]
0 - state: [-0.4529439  -0.00310886]
0 - state: [-0.45557868 -0.00263481]
0 - state: [-0.4597201  -0.00414143]
0 - state: [-0.4653377  -0.00561759]
0 - state: [-0.47039002 -0.00505233]
0 - state: [-0.47683972 -0.0064497 ]
0 - state: [-0.48263898 -0.00579924]
0 - state: [-0.48774466 -0.00510567]
0 - state: [-0.49411872 -0.00637406]
0 - state: [-0.49971357 -0.00559487]
0 - state: [-0.50448745 -0.00477386]
0 - state: [-0.5104045  -0.00591711]
0 - state: [-0.5154206  -0.00501604]
0 - state: [-0.519498   -0.00407737]
0 - state: [-0.5226061  -0.00310813]
0 - state: [-0.52572167 -0.00311557]
0 - state: [-0.52982134 -0.00409965]
0 - state: [-0.5328743  -0.00305299]
0 - state: [-0.5368577  -0.00398343]
0 - state: [-0.54074174 -0.00388401]
0 - state: [-0.54449725 -0.0037555 ]
0 - state: [-0.5470961  -0.00259886]
0 - state: [-0.5505189  -0.00342278]
0 - state: [-0.55473995 -0.0042211 ]
0 - state: [-0.55972785 -0.00498788]
0 - state: [-0.5654453  -0.00571744]
0

In [2]:
env = gym.make("MountainCar-v0", render_mode='human')

# Break POS and Velocity into 20 segments
pos_space = np.linspace(env.observation_space.low[0], env.observation_space.high[0], 20) 
vel_space = np.linspace(env.observation_space.low[1], env.observation_space.high[1], 20)

for i in range(3):

    state = env.reset()[0]
    # find segment of agents POS and Velocity
    state_p = np.digitize(state[0], pos_space)
    state_v = np.digitize(state[1], vel_space)

    terminated = False
    rewards = 0

    while(not terminated and rewards > -1000):
        action = np.argmax(q[state_p, state_v, :])

        new_state,reward,terminated,_,_ = env.step(action)
        new_state_p = np.digitize(state[0], pos_space)
        new_state_v = np.digitize(state[1], vel_space)

        state = new_state
        state_p = new_state_p
        state_v = new_state_v

        rewards += reward

    epsilon = max(epsilon - epsilon_decay_rate, 0)

    print(f'Episode: {i+1} - Steps: {rewards*-1}')

env.close()

Episode: 1 - Steps: 1000.0
Episode: 2 - Steps: 296.0
Episode: 3 - Steps: 1000.0
